In [38]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"

In [39]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [40]:
def grade_by_model(test_case, output):
    # Create evaluation prompt
    eval_prompt = """
    You are an expert code reviewer. Evaluate this AI-generated solution.
    
    Task: {task}
    Solution: {solution}
    
    Provide your evaluation as a structured JSON object with:
    - "strengths": An array of 1-3 key strengths
    - "weaknesses": An array of 1-3 key areas for improvement  
    - "reasoning": A concise explanation of your assessment
    - "score": A number between 1-10
    """
    
    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [41]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])

    return json.loads(text)

In [42]:
import json

dataset = generate_dataset()
print(json.dumps(dataset, indent=2))

[
  {
    "task": "Create a Python function to extract the AWS account ID from an AWS ARN (Amazon Resource Name)"
  },
  {
    "task": "Write a JSON schema to validate an AWS EC2 instance configuration with required fields like instance type, AMI ID, and security group"
  },
  {
    "task": "Develop a regular expression to match and validate AWS Lambda function names, which must start with a letter, can contain letters, numbers, and hyphens, and be between 1-64 characters long"
  }
]


In [43]:
with open('dataset.json', 'w') as f:
    json.dump(dataset, f, indent=2)

In [44]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""
    
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [45]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)
    
    # TODO - Grading
    # Grade the output
    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]
    
    return {
        "output": output,
        "test_case": test_case,
        "score": score
    }

In [46]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []
    
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)
    
    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")
    
    return results

In [47]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

print(json.dumps(results, indent=2))

Average score: 3.3333333333333335
[
  {
    "output": "Here's a Python function to extract the AWS account ID from an AWS ARN:\n\n```python\ndef extract_account_id_from_arn(arn):\n    \"\"\"\n    Extract the AWS account ID from a given ARN.\n    \n    Args:\n        arn (str): The full AWS ARN string\n    \n    Returns:\n        str: The AWS account ID, or None if the ARN is invalid\n    \"\"\"\n    # Check if the ARN is a valid string\n    if not isinstance(arn, str):\n        return None\n    \n    # Split the ARN into its components\n    # Typical ARN format: arn:aws:service:region:account-id:resource-type/resource-id\n    try:\n        arn_parts = arn.split(':')\n        \n        # Validate basic ARN structure\n        if len(arn_parts) < 5 or arn_parts[0] != 'arn' or arn_parts[1] != 'aws':\n            return None\n        \n        # The account ID is typically the 5th part of the ARN\n        account_id = arn_parts[4]\n        \n        # Validate that the account ID is a 12-di